In [1]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import plotly.graph_objects as pgo


In [2]:
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn

In [6]:
#Load the dataset
data = pd.read_csv('Churn_Modeling.csv')

data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
# Specify a list of categorical columns to be converted
categorical_columns = ['Geography', 'Gender']

# Convert categorical columns to numerical using one-hot encoding
data = pd.get_dummies(data, columns=categorical_columns)

# Fill missing values with mode (most frequent value) for each column
data = data.fillna(data.mode().iloc[0])

# Now, your categorical columns are converted to numerical columns
# and missing values are filled with the mode.
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1,15634602,Hargrave,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,2,15647311,Hill,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,3,15619304,Onio,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,4,15701354,Boni,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,5,15737888,Mitchell,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
5,6,15574012,Chu,645,44,8,113755.78,2,1,0,149756.71,1,0,0,1,0,1
6,7,15592531,Bartlett,822,50,7,0.00,2,1,1,10062.80,0,1,0,0,0,1
7,8,15656148,Obinna,376,29,4,115046.74,4,1,0,119346.88,1,0,1,0,1,0
8,9,15792365,He,501,44,4,142051.07,2,0,1,74940.50,0,1,0,0,0,1
9,10,15592389,H?,684,27,2,134603.88,1,1,1,71725.73,0,1,0,0,0,1


In [8]:
data['EstimatedSalary'] = data['EstimatedSalary'].astype(float)
data['Balance'] = data['Balance'].astype(float)


In [9]:
# X contains the features (all columns except 'Exited')
X = data.drop(columns=['Exited','Surname'])
# y contains the target variable 'Exited'
y = data['Exited']

In [10]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy=0.4)
X, y = smote.fit_resample(X, y)

In [11]:
from sklearn.model_selection import train_test_split

# Split the data into 80% training and 20% testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 'X_train' and 'y_train' are the features and target variable for the training set
# 'X_test' and 'y_test' are the features and target variable for the testing set

In [12]:
data.columns.values

array(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female',
       'Gender_Male'], dtype=object)

In [15]:
from sklearn.linear_model import LogisticRegression

# Build Logistic Regression model
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

# Make predictions
lr_predictions = lr_model.predict(X_test)

# Evaluate Logistic Regression
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_predictions))
print("\nClassification Report:\n", classification_report(y_test, lr_predictions))


Logistic Regression Accuracy: 0.7183856502242153

Classification Report:
               precision    recall  f1-score   support

           0       0.72      1.00      0.84      1602
           1       0.00      0.00      0.00       628

    accuracy                           0.72      2230
   macro avg       0.36      0.50      0.42      2230
weighted avg       0.52      0.72      0.60      2230



C:\Users\kxion\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kxion\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kxion\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Fit the model
rf_model.fit(X_train, y_train)

# Access feature importances
feature_importances = rf_model.feature_importances_

# Create a DataFrame to display feature names and their importances
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the sorted DataFrame
print(feature_importance_df)


              Feature  Importance
3                 Age    0.205585
6       NumOfProducts    0.102672
5             Balance    0.092689
1          CustomerId    0.084988
0           RowNumber    0.084742
9     EstimatedSalary    0.083968
2         CreditScore    0.083963
8      IsActiveMember    0.055946
4              Tenure    0.051651
13      Gender_Female    0.034162
10   Geography_France    0.031618
14        Gender_Male    0.031284
12    Geography_Spain    0.025465
11  Geography_Germany    0.018033
7           HasCrCard    0.013233
